In [3]:
import pandas as pd
import networkx as nx

In [45]:
def build_graph(file_path):
    graph_data_frame = pd.read_csv(file_path, sep='\t')
    graph_data_frame['PARENTID'].fillna('X', inplace=True)

    G = nx.DiGraph(world=graph_data_frame['world'][0])
    edge_list = list(zip(graph_data_frame['PARENTID'], graph_data_frame['NID']))
    edge_list.pop(0) # Remove edge leading to root node.
    G.add_edges_from(edge_list)

    name_dict = {node: name for node, name in
                          zip(graph_data_frame['NID'], graph_data_frame['NID'])
                          }
    node_location_dict = {node: location for node, location in
                          zip(graph_data_frame['NID'], graph_data_frame['node_location'])
                          }
    depth_dict = {node: depth for node, depth in 
                  zip(graph_data_frame['NID'], graph_data_frame['depth'])
                          }
    new_observations_dict = {node: new_observations for node, new_observations in 
                             zip(graph_data_frame['NID'], graph_data_frame['new_observations'])
                          }
    steps_from_root_dict = {node: steps for node, steps in 
                            zip(graph_data_frame['NID'], graph_data_frame['steps_from_root'])
                          }
    is_leaf_dict = {node: truth_value for node, truth_value in 
                    zip(graph_data_frame['NID'], graph_data_frame['is_leaf'])
                          }
    path_value_dict = {node: value for node, value in 
                       zip(graph_data_frame['NID'], graph_data_frame['node_value'])
                          }
    node_ep_dict = {node: node_ep for node, node_ep in 
                    zip(graph_data_frame['NID'], graph_data_frame['node_ep'])
                          }
    black_remains_dict = {node: black_remains for node, black_remains in 
                          zip(graph_data_frame['NID'], graph_data_frame['black_remains'])
                          }
    steps_from_parent_node_dict = {node: steps for node, steps in 
                          zip(graph_data_frame['NID'], graph_data_frame['steps_from_parent'])
                          }
    choice_direction_dict = {node: direction for node, direction in
                          zip(graph_data_frame['NID'], [get_choice_direction(path) for path in graph_data_frame['path_from_parent']])
                          }
    
    nx.set_node_attributes(G, name_dict, 'name')
    nx.set_node_attributes(G, node_location_dict, 'node_location')
    nx.set_node_attributes(G, depth_dict, 'depth')
    nx.set_node_attributes(G, new_observations_dict, 'new_observations')
    nx.set_node_attributes(G, steps_from_root_dict, 'steps_from_root')
    nx.set_node_attributes(G, is_leaf_dict, 'is_leaf')
    nx.set_node_attributes(G, path_value_dict, 'path_value')
    nx.set_node_attributes(G, node_ep_dict, 'node_ep')
    nx.set_node_attributes(G, black_remains_dict, 'black_remains')
    nx.set_node_attributes(G, steps_from_parent_node_dict, 'steps_from_parent')
    nx.set_node_attributes(G, choice_direction_dict, 'choice_direction')

    path_from_parent_dict = {edge: path for edge, path in 
                             zip(edge_list, graph_data_frame['path_from_parent'][1:])
                            }
    steps_from_parent_edge_dict = {edge: steps for edge, steps in 
                             zip(edge_list, graph_data_frame['steps_from_parent'][1:])
                            }

    nx.set_edge_attributes(G, path_from_parent_dict, 'path_from_parent')
    nx.set_edge_attributes(G, steps_from_parent_edge_dict, 'steps_from_parent')
    nx.set_edge_attributes(G, steps_from_parent_edge_dict, 'weight') # Assign steps from parent to node as edge weight
    
    return G

In [46]:
def parse_path_string(path_string):
    a = path_string.replace('p','')
    b = a.split(';')
    b.remove('')
    
    return b

In [47]:
def parse_location(location: str):
    location_list = location
    location_list = location_list.strip('()')
    location_list = location_list.split(',')
    location_list = [int(item) for item in location_list]
    
    return location_list

In [48]:
def get_choice_direction(path_from_parent):
    parsed_path = parse_path_string(path_from_parent)
    
    try:
        current_location = parse_location(parsed_path[0])
        next_location = parse_location(parsed_path[1])
    except IndexError:
        return None
    
    # Left
    if next_location[0] < current_location[0]:
        return 0
    # Right
    elif next_location[0] > current_location[0]:
        return 1
    # Up
    elif next_location[1] < current_location[1]:
        return 2
    # Down
    elif next_location[1] > current_location[1]:
        return 3
    else:
        raise Exception("No direction could be assigned. Check function input")

In [8]:
parse_path_string('p(3,4);p(3,3);p(3,2);')[1]

'(3,3)'

In [10]:
parse_location(parse_path_string('p(3,4);p(3,3);p(3,2);')[1])

[3, 3]

In [41]:
get_choice_direction('p(0,0);')

In [50]:
H = build_graph('treebuilderUpdated/courtyard.csv')

In [51]:
nx.get_node_attributes(H, 'choice_direction')

{'N6609': None,
 'N2554': 1,
 'N2200': 3,
 'N6588': 3,
 'N5553': 0,
 'N6173': 1,
 'N7822': 3,
 'N3405': 1,
 'N5323': 2}